In [1]:
import  pandas as pd

In [2]:
file10 = "csv-origins/GE2010_byCounty.csv"
file14 = "csv-origins/GE2014_byCounty.csv"
file18 = "csv-origins/GE2018_byCounty.csv"
ge10 = pd.read_csv(file10)
ge14 = pd.read_csv(file14)
ge18 = pd.read_csv(file18)

# Cleaning 2010 Results

In [3]:
k10, n10 = ge10.keys(), ge10.size

In [4]:
#check out the value counts for each column
valno10 = dict.fromkeys(k10)
for k in k10:
    valno10[k] = getattr(ge10,k).value_counts()

In [5]:
#these columns are not useful
dropvar10 = ["CanAffilCommit", "Election", "VoteFor", "PartyAbbrev","OfficeSequence"]

In [6]:
#so lets drop them
ge10.drop(columns=dropvar10,inplace=True)

# Cleaning 2014 Results

In [7]:
#start the same as with 2010
k14, n14 = ge14.keys(), ge14.size

In [8]:
valno14 = dict.fromkeys(k14)
for k in k14:
    valno14[k] = getattr(ge14,k).value_counts()

In [9]:
#drop the same as in 2010, also new column ID
dropvar14 = dropvar10 + ['ID']

In [10]:
ge14.drop(columns=dropvar14,inplace=True)

In [11]:
#this is so I can sort the frame by most important ballot group, like how 2010 is sorted by default
bgdict = dict(ge14.BallotGroup.value_counts())
bgcount = []
for bg in ge14.BallotGroup:
    bgcount.append(bgdict[bg])
bgser = pd.Series(bgcount,name="BGCount")

In [12]:
ge14 = ge14.merge(bgser,left_index=True,right_index=True)

In [13]:
ge14.sort_values(by=["BGCount","CandidateID","Votes"],ascending=[False,True,False],inplace=True,ignore_index=True)

In [14]:
ge14.drop(columns="BGCount",inplace=True)

## Instead of making one giant multiindex, I thought it made sense to split the dataset into three tables to take advantage of primary and foreign keys.
## Frame "ge14" will split into "candidates14", "ballotgroups14", and "votes14"
### Frame "ballotgroup14" contains a new column, "NumberRunning", counting how many candidates there are for each office.

In [15]:
candidates14 = ge14[["CandidateID","CanFirstName", "CanLastName","PartyName","OfficeName"]].copy()
candidates14.drop_duplicates(inplace=True)
candidates14.set_index(candidates14.CandidateID,inplace=True)
candidates14.drop(columns="CandidateID",inplace=True)
candidates14.sort_values(by="CandidateID")

,CanFirstName,CanLastName,PartyName,OfficeName
CandidateID,,,,
19443,CHAPIN,ROSE,REPUBLICAN,51ST SENATE
19444,CHRIS E.,REIF,REPUBLICAN,7TH CIRCUIT - MITCHELL VACANCY
19445,MIKE,ATTERBERRY,REPUBLICAN,8TH CIRCUIT - POPE VACANCY
19446,THOMAS B.,EWING,DEMOCRATIC,9TH CIRCUIT - DANNER VACANCY
19447,DAVE,CARLSON,REPUBLICAN,12TH CIRCUIT - 1ST SUBCIRCUIT - A VACANCY
...,...,...,...,...
20242,HILAIRE F.,SHIOURA,NaN,UNITED STATES SENATOR
20243,PHIL,COLLINS,NaN,9TH CONGRESS
20244,TOM,DEMAS,NaN,78TH REPRESENTATIVE


In [16]:
runningno = ge14.OfficeName.value_counts()
rndict = dict(runningno)
rnlist = []
for o in ge14.OfficeName:
    rnlist.append(rndict[o])
NumberRunning = pd.Series(rnlist,name="NumberRunning")

In [17]:
bg14 = ge14[["OfficeName","BallotGroup"]].copy()
bg14 = bg14.merge(NumberRunning,left_index=True, right_index=True)
bg14.drop_duplicates(subset="OfficeName",inplace=True)
indx = pd.MultiIndex.from_frame(bg14[["BallotGroup","OfficeName"]])
bg14.set_index(indx,inplace=True)

In [18]:
bg14.drop(columns=["BallotGroup","OfficeName"],inplace=True)

In [19]:
bglist2 =[]
for bg in [bg14.index[i][0] for i in range(407)]:
    bglist2.append(bgdict[bg])
bgdf2 = pd.DataFrame(bglist2,index=indx)
bg14 = bg14.merge(bgdf2,left_index=True,right_index=True)

In [20]:
bg14.sort_values(by=[0,"NumberRunning"],ascending=[False,False],inplace=True)
ballotgroups14 = bg14.drop(columns=0)

In [21]:
votes14 = ge14[["CandidateID","County","Votes"]].copy()
indx2 = pd.MultiIndex.from_frame(ge14[["CandidateID","County"]])
votes14.set_index(indx2,inplace=True)
votes14.drop(columns=["CandidateID","County"],inplace=True)
votes14.sort_values(by=['CandidateID','Votes'],ascending=[True,False],inplace=True)

In [22]:
candidates14.head()

,CanFirstName,CanLastName,PartyName,OfficeName
CandidateID,,,,
19748,PAUL M.,SCHIMPF,REPUBLICAN,ATTORNEY GENERAL
19749,LISA,MADIGAN,DEMOCRATIC,ATTORNEY GENERAL
19751,JUDY BAAR,TOPINKA,REPUBLICAN,COMPTROLLER
19752,SHEILA,SIMON,DEMOCRATIC,COMPTROLLER
19762,BRUCE,RAUNER,REPUBLICAN,GOVERNOR AND LIEUTENANT GOVERNOR


In [23]:
ballotgroups14.head()

NumberRunning
BallotGroup OfficeName                                     
B           GOVERNOR AND LIEUTENANT GOVERNOR            761
            COMPTROLLER                                 403
            SECRETARY OF STATE                          334
            ATTORNEY GENERAL                            306
            TREASURER                                   306

In [24]:
votes14.head()

Votes
CandidateID County          
19443       MACON      16658
            CHAMPAIGN  15501
            SHELBY      7116
            PIATT       6443
            DOUGLAS     5623

# Export tables to csvs

In [25]:
candidates14.to_csv("csv-clean/candidates14.csv",index_label="CandidateID")
ballotgroups14.to_csv("csv-clean/ballotgroups14.csv",index_label=["BallotGroup","OfficeName"])
votes14.to_csv("csv-clean/votes14.csv",index_label=["CandidateID","County"])

# Will continue in the same way with GE18 and GE10 unless otherwise told!

In [31]:
k18, n18 = ge18.keys(), ge18.size

In [34]:
valno18 = dict.fromkeys(k18)
for k in k18:
    valno18[k] = getattr(ge18,k).value_counts()

In [35]:
dropvar18 = dropvar14